# Annotation

- annotation은 코드 사용자에게 함수 인자로 어떤 값이 와야 하는지 `힌트`를 주는 것이다.

In [7]:
# 어노테이션을 이용한 변수의 예상 타입 지정
from dataclasses import dataclass

@dataclass
class Point:
    lat: float
    long: float

def locate(latitude: float, longitude: float) -> Point: #함수 반환 값 예상 타입 지정
    """맵에서 좌표에 해당하는 객체를 검색"""

- latitude, longitude는 float 타입의 변수이다.
- locate 함수는 Point 인스턴스를 반환하는 것 또한 예상할 수 있다.
- 이를 통해 사용자는 특정 함수에 필요한 변수의 타입에 대한 정보를 얻을 수 있다.
- 하지만 어노테이션은 말 그대로 힌트를 줄 뿐 타입을 강제하지 않는다.
- **어노테이션은 타입 뿐만 아니라 파이썬 인터프리터에서 유효한 모든 것을 사용할 수 있다.**
    - 변수의 의도를 설명하는 문자열, 콜백이나 유효성 검사 함수로 사용할 수 있는 callable


delay_in_seconds 파라미터는 긴 이름을 가지고 있어 많은 정보를 제공하는 것 같지만 다음의 정보가 부족하다.
- 허용 가능한 시간은?
- 분수를 입력해도 되나?
- 코드를 이렇게 수정하면 어떻게 되나?

```
def launch_task(delay_in_seconds):
    ...
```

어노테이션을 사용하면 `__annotation__` 이라는 특수한 속성이 생긴다.
- 어노테이션의 이름과 값을 맵핑한 사전 타입의 값이다.

In [15]:
def launch_task(delay: "please pass second variable(int)") -> "wait for {delay} seconds":
    pass

launch_task.__annotations__

{'delay': 'please pass second variable(int)',
 'return': 'wait for {delay} seconds'}

In [11]:
def launch_task(delay: "please pass second variable(int)"):
    pass

launch_task.__annotations__

{'delay': 'please pass second variable(int)'}

## 어노테이션의 명시적 표현

- 타입 힌트는 단순히 데이터 타입을 알려주는 것 뿐만 아니라 적절한 데이터 타입 추상화를 하도록 도와줄 수 있다.
- 클라이언트 배열에서 어떤 작업을 수행하는 함수가 있다고 가정해보자.

In [22]:
def process_clients(clients: list):
    pass

print(process_clients.__annotations__)

{'clients': <class 'list'>}


- clients라는 이름의 변수가 list 타입이라는 힌트를 얻을 수 있다.
- 여기서 그치지 않고 더 많은 정보를 주기 위해 아래와 같이 코드를 수정할 수 있다.

In [27]:
def process_clients(clients: list[tuple[int, str]]):
    pass

print(process_clients.__annotations__)

{'clients': list[tuple[int, str]]}


- clients라는 변수가 어떤 데이터 타입인지 확인할 수 있지만 아래와 같이 데이터 구조를 따로 정의하여 명시적으로 어노테이션을 작성하는 것이 바람직하다.
- 아래와 같이 어노테이션을 작성하는 경우 나중에 다른 객체나 클래스로 변경해도 어노테이션을 그대로 사용할 수 있다는 장점이 있다.
- 이러한 방법의 취지는 유의미한 문법 구조를 통해 코드의 의미나 의도를 더욱 쉽게 이해할 수 있도록 하자는 것이다.

In [35]:
from typing import Tuple
Client = Tuple[int, str]
def process_clients(cleints: list[Client]):
    pass

print(process_clients.__annotations__)

{'cleints': list[typing.Tuple[int, str]]}


### @dataclass 데코레이터를 이용한 자동 속성 할당

- PEP-526, PEP-557 표준을 도입하면 클래스를 보다 간결하게 작성하고 작은 컨테이너 객체를 쉽게 정의할 수 있다.
- 클래스에서 데이터 타입에 대한 어노테이션과 함게 속성을 선언하고 @dataclass 데코레이터를 이용하면 별도의 __init__ 메소드에서 변수를 선언하고 할당하는 작업을 하지 않아도 바로 인스턴스 속성으로 인식한다.

In [44]:
# Before
class Point:
    def __init__(self, lat, long):
        self.lat = lat
        self.long = long

p1 = Point(10, 20)
print(f'lat : {p1.lat} | long : {p1.long}\n')

# After
# __init__ 메소드 미사용
from dataclasses import dataclass
@dataclass
class Point:
    lat: float
    long: float

p2 = Point(10, 20)
print(p2.__annotations__)
print(f'lat : {p1.lat} | long : {p1.long}')


lat : 10 | long : 20

{'lat': <class 'float'>, 'long': <class 'float'>}
lat : 10 | long : 20


## 어노테이션은 docstrig을 대체하는가?

- docstring에 포함된 정보의 일부는 어노테이션으로 이동시킬 수 있다.
- 동적 데이터 타입과 중첩 데이터 타입의 경우 예상 데이터 예제를 제공하여 어떤 형태의 데이터를 다루는지 제공하는 것이 좋다.

다음과 같은 데이터 유효성 검사 함수가 있다고 가정해보자.

- **알 수 있는 정보**
    - 사전 형태의 파라미터를 받아 사전 형태의 값을 반환
- **알 수 없는 정보**
    - response 객체의 올바른 인스턴스는 어떤 형태인가?
    - 결과의 인스턴스는 어떤 형태인가?

위의 두가지 질문에 모두 대답하기 위해선 docstring을 보완하여 작성하는 것이 바람직하다.

In [45]:
# 데이터 유효성 검사 함수
def data_from_response(response: dict) -> dict:
    if response["status"] != 200:
        raise ValueError
    return {"data": response["payload"]}


# docstring 작성
def data_from_response(response: dict) -> dict:
    """response의 HTTP status가 200이라면 response의 payload를 반환
    - response 사전의 예제::
    {
        "status": 200, # <int>
        "timestamp": "...", # 현재 시간의 ISO 포맷 문자열
        "payload": {...} # 반환하려는 사전 데이터
    }
    - 반환 사전 값의 예제::
    {"data": {...}}
    - 발생 가능한 예외:
    - HTTP status가 200이 아닌 경우 ValueError 발생
    """
    if response["status"] != 200:
        raise ValueError
    return {"data": response["payload"]}

print(data_from_response.__doc__)

response의 HTTP status가 200이라면 response의 payload를 반환
    - response 사전의 예제::
    {
        "status": 200, # <int>
        "timestamp": "...", # 현재 시간의 ISO 포맷 문자열
        "payload": {...} # 반환하려는 사전 데이터
    }
    - 반환 사전 값의 예제::
    {"data": {...}}
    - 발생 가능한 예외:
    - HTTP status가 200이 아닌 경우 ValueError 발생
    


## 데이터 타입 일관성 검사 (mypy)
```
$ pip install mypy
```
- 가상환경에 mypy를 설치하고 mypy [파일명] 명령어를 실행하면 의심이 되는 오류를 반환한다.
- mypy의 오류 탐지가 잘못된 탐지로 생각되면 다음과 같이 해당 라인에 대한 검사를 무시하도록 할 수 있다.
```
type_to_ignore = "something" # type: ignore
```
- mypy나 기타 타입 검사 도구가 유용하게 쓰이기 위한 사전 조건은 `어노테이션`을 정확하게 작성하는 것이다.
- 예를 들어 다음과 같은 코드는 mypy에서 오류를 보고하지 않을 것이다.
```
def broadcast_notification(
    message: str,
    relevant_user_emails = Iterable[str]

    for email in relevant_user_emails:
        logger.info(f"{message} 메세지를 {email}에게 전달")
)

broadcast_notification("welcome", "user1@domain.com)
```
위 코드의 경우 String역시 Iterable하므로 mypy에서 오류를 반환하지 않고 잘못된 코드가 실행 될 것이다.
